In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# Hyper parameters
num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001

In [2]:
trainset = torchvision.datasets.MNIST(root='./mnist', train=True,
                                        download=True, transform=transforms.ToTensor())
print("train : " + str(len(trainset)) + ' images')

testset = torchvision.datasets.MNIST(root='./mnist', train=False,
                                        download=True, transform=transforms.ToTensor())
print("test : " + str(len(testset)) + ' images')

# Data loader
trainloader = torch.utils.data.DataLoader(dataset=trainset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

testloader = torch.utils.data.DataLoader(dataset=testset, 
                                          batch_size=batch_size, 
                                          shuffle=False)



train : 60000 images
test : 10000 images


In [3]:
# Convolutional neural network (two convolutional layers)
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    
    #define the learnable paramters by calling the respective modules (nn.Conv2d, nn.MaxPool2d etc.)
    def __init__(self):
        super(Net, self).__init__()
        
        #calling conv2d module for convolution
        self.layer1 = nn.Sequential(
                        nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
                        nn.BatchNorm2d(16),
                        nn.ReLU(),
                        nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
                        nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
                        nn.BatchNorm2d(32),
                        nn.ReLU(),
                        nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.fc=nn.Linear(32*7*7,num_classes)
        
    
    #defining the structure of the network
    def forward(self, x):
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = x.reshape(x.size(0), -1)
        out = self.fc(x)
        return out#,resp2,resp2,resp4
                        

model = Net()
if torch.cuda.is_available():
    model = model.cuda()

#Printing the network architecture
print(model)



Net(
  (layer1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=1568, out_features=10, bias=True)
)


In [12]:
model.fc.weight

Parameter containing:
tensor([[-0.0191, -0.0233,  0.0153,  ..., -0.0242, -0.0243,  0.0151],
        [-0.0155,  0.0071,  0.0111,  ...,  0.0135,  0.0095,  0.0186],
        [-0.0133, -0.0251,  0.0213,  ..., -0.0158,  0.0126,  0.0239],
        ...,
        [ 0.0197, -0.0050,  0.0043,  ..., -0.0096, -0.0039,  0.0139],
        [-0.0061, -0.0199,  0.0120,  ..., -0.0006,  0.0088,  0.0148],
        [-0.0104,  0.0186, -0.0119,  ...,  0.0039,  0.0214,  0.0003]],
       device='cuda:0', requires_grad=True)

In [ ]:

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  


In [ ]:
########################################################################
# Train the network
# ^^^^^^^^^^^^^^^^^^^^

def train(epoch, trainloader, optimizer, criterion):
    running_loss = 0.0
    
    for i, (inputs,labels) in enumerate(tqdm(trainloader), 0):
        # get the inputs
        inputs = inputs.to(device)
        labels = labels.to(device)
            
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # addup loss
        running_loss += loss.item()

    print('epoch %d training loss: %.3f' %
            (epoch + 1, running_loss / (len(trainloader))))
    return running_loss / (len(trainloader))   

In [ ]:
########################################################################
# Let us look at how the network performs on the test dataset.

def test(testloader, model):
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for (inputs,labels) in tqdm(testloader):
            # get the inputs
            inputs = inputs.to(device)
            labels = labels.to(device)

            # forward + backward + optimize
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # addup loss
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the %d test images: %d %%, loss = %f' % (
                                    len(testset),100 * correct / total, running_loss / len(testloader)))
    return running_loss / len(testloader)


In [ ]:
import os
from tqdm import tqdm
print('Start Training')
# if not os.path.exists('./models'):
#     os.mkdir('./models')

training_losses = []
testing_losses = []

for epoch in range(num_epochs):  # loop over the dataset multiple times
    print('epoch ', epoch + 1)
    train_loss = train(epoch, trainloader, optimizer, criterion)
    test_loss = test(testloader, model)
#     classwise_test(testloader, net)
    
    
    training_losses.append(train_loss)
    testing_losses.append(test_loss)

print('Finished Training')


In [ ]:
# running_loss=0
# dataiter = iter(testloader)
# inputs, labels = dataiter.next()
# inputs=inputs.to(device)
# labels=labels.to(device)
# outputs = model(inputs)
# loss = criterion(outputs, labels)
# running_loss += loss.item()

# correct=(predicted==labels).sum().item()
# total=inputs.size(0)
# a,pred=torch.max(outputs,1)
# pred


In [ ]:
# #Torch tensor
# x=torch.randn(2,3)
# print(x)
# print(type(x))

# #Torch tensor to numpy
# xnp=x.numpy()
# print(xnp)
# print(type(xnp))

# #Numpy to Torch tensor
# xx=torch.from_numpy(xnp)
# print(xx)
# print(type(xx))

In [ ]:
# ## Transposing axis

# from skimage import io
# np_img = io.imread('./data/ferrari.png')
# print(np_img.shape)

# np_img = np_img.transpose((2,0,1))
# print(np_img.shape)

# torch_img = torch.from_numpy(np_img)
# print(torch_img.shape)

In [ ]:
# images, labels = iter(dataloader).next()
# print(type(images))

#Unsqueeze
# unsqueezed_img = images[0].unsqueeze(0)
# print(unsqueezed_img.shape)

# #Squeeze
# squeezed_img = images[0].squeeze(0)
# print(squeezed_img.shape)